In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
import pickle


In [4]:
data = pd.read_csv('/content/cleaned_plagiarism_dataset_csv.csv')

In [5]:
df = pd.DataFrame(data)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df['source_text'], df['plagiarized_text'], df['label'], test_size=0.2, random_state=42)

# Step 1: TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

# Step 2: Train SVM Model
model = SVC(probability=True)  # (probability=True if you want predict_proba, optional)
model.fit(X_train_tfidf, y_train)


ValueError: too many values to unpack (expected 4)

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score

# ... (load your data as before) ...
data = pd.read_csv('/content/cleaned_plagiarism_dataset_csv.csv')

df = pd.DataFrame(data)

# Step 1: Create a similarity feature
vectorizer = TfidfVectorizer()
tfidf_source = vectorizer.fit_transform(df['source_text'])
tfidf_plagiarized = vectorizer.transform(df['plagiarized_text'])
df['similarity'] = [cosine_similarity(tfidf_source[i], tfidf_plagiarized[i])[0][0] for i in range(len(df))]

# Step 2: Split the data
X_train, X_test, y_train, y_test = train_test_split(df[['similarity']], df['label'], test_size=0.2, random_state=42)

# Step 3: Train SVM Model
model = SVC(probability=True)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)  # Assuming you've trained the model using Approach 1 or 2

# Step 5: Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.584


In [13]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
df = pd.read_csv('/content/cleaned_plagiarism_dataset_csv.csv')

# Initialize BERT model
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute BERT-based embeddings and semantic similarity
df['source_embed'] = df['source_text'].apply(lambda x: bert_model.encode(x, convert_to_numpy=True))
df['plag_embed'] = df['plagiarized_text'].apply(lambda x: bert_model.encode(x, convert_to_numpy=True))
df['semantic_similarity'] = df.apply(
    lambda row: cosine_similarity([row['source_embed']], [row['plag_embed']])[0][0], axis=1
)

# Jaccard similarity
def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

df['jaccard'] = df.apply(
    lambda row: jaccard_similarity(row['source_text'], row['plagiarized_text']), axis=1
)

# Length difference
df['length_diff'] = abs(df['source_text'].str.len() - df['plagiarized_text'].str.len())

# Feature set and label
features = df[['semantic_similarity', 'jaccard', 'length_diff']]
labels = df['label']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42
)

# Train XGBoost model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)

# Prediction and evaluation
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Accuracy: 0.944
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.92      0.94       115
           1       0.94      0.96      0.95       135

    accuracy                           0.94       250
   macro avg       0.95      0.94      0.94       250
weighted avg       0.94      0.94      0.94       250



/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [09:41:22] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [14]:
with open('xgb_model.pkl', 'wb') as f:
    pickle.dump(model, f)

bert_model.save('./bert_model')  # Save BERT model directory

# --------------------------
# Step 2: Later, Load and Predict on New Sample
# Load model
with open('xgb_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

In [15]:
!zip -r /content/bert_model.zip /content/bert_model
from google.colab import files
files.download('/content/bert_model.zip')

  adding: content/bert_model/ (stored 0%)
  adding: content/bert_model/1_Pooling/ (stored 0%)
  adding: content/bert_model/1_Pooling/config.json (deflated 57%)
  adding: content/bert_model/config.json (deflated 48%)
  adding: content/bert_model/tokenizer.json (deflated 71%)
  adding: content/bert_model/special_tokens_map.json (deflated 80%)
  adding: content/bert_model/README.md (deflated 64%)
  adding: content/bert_model/2_Normalize/ (stored 0%)
  adding: content/bert_model/tokenizer_config.json (deflated 73%)
  adding: content/bert_model/modules.json (deflated 62%)
  adding: content/bert_model/sentence_bert_config.json (deflated 4%)
  adding: content/bert_model/vocab.txt (deflated 53%)
  adding: content/bert_model/model.safetensors (deflated 9%)
  adding: content/bert_model/config_sentence_transformers.json (deflated 34%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>